v1.2

- The objective of this **01.06** notebook is to 
  - calculate the average word2vec value per sentence

In [132]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    print("Not a Goolge Drive Environment.")

Not a Goolge Drive Environment.


In [133]:
import time
t_start = time.time()

# Setup Environment

## Install Modules

In [134]:
#!pip install watermark
#!pip install gensim
#!pip install fasttext

In [135]:
import nltk
nltk.download('popular')
nltk.download('stopwords')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/busera/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/busera/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/busera/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/busera/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/busera/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/busera/nltk_data...
[nltk_data]    |   Package movie_reviews is already

True

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/busera/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Import Modules

In [136]:
# Base libraries
import os
import re
import platform

# Scientific libraries
import numpy as np
import pandas as pd

# Visualization
import seaborn as sns
sns.set(rc={'figure.figsize':(8,4)})
sns.set(font_scale=0.8)

# Helper libraries
from tqdm import tqdm
tqdm.pandas()
from watermark import watermark
import gc # garbage collection to optimize memory usage, use gc.collect()
import warnings
warnings.filterwarnings('ignore')

# Pandas options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Load magic commands
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [137]:
# Specific libraries
import gensim
import nltk
from nltk.stem import WordNetLemmatizer
import gensim.downloader as gensim_api
import fasttext as ft

## Define Parameters

In [138]:
# None

### Helper Functions

In [139]:
import http.client, urllib

def send_push(message):
	"""Send push notifications to pushover service."""
	conn = http.client.HTTPSConnection("api.pushover.net:443")
	conn.request("POST", "/1/messages.json",
	urllib.parse.urlencode({
		"token": "ahs1q4mwpnxe3645zeaqzas69whq7a",  # ML Notifications Channel
		"user": "u5vr1qkc9ghudg2ehuug153okeiz1d",
		"message": message,
	}), { "Content-type": "application/x-www-form-urlencoded" })

	conn.getresponse()

# Load Data

In [140]:
try:
    # Location for "shared with" people
    # create a shortcut of the shared folder in your Google Drive root folder
    PATH_DATA = "/content/drive/MyDrive/SIADS696/Environment/data/"
    PATH_DATA_RAW = "/content/drive/MyDrive/SIADS696/Environment/data/raw/"
    PATH_DATA_INT = "/content/drive/MyDrive/SIADS696/Environment/data/interim/"
    PATH_DATA_PRO = "/content/drive/MyDrive/SIADS696/Environment/data/processed/"
    PATH_DATA_MOD = "/content/drive/MyDrive/SIADS696/Environment/models/"

    df_wiki_train = pd.read_parquet(PATH_DATA_INT + "train_features_clean_stats.parquet.gzip")
    df_wiki_test = pd.read_parquet(PATH_DATA_INT + "test_features_clean_stats.parquet.gzip")

except:
    print("Not a Goolge Drive Environment. Loading local files.")
    PATH_DATA = "../data/"
    PATH_DATA_RAW = "../data/raw/"
    PATH_DATA_INT = "../data/interim/"
    PATH_DATA_PRO = "../data/interim/"
    PATH_DATA_MOD = "../models/"
    
    df_wiki_train = pd.read_parquet(PATH_DATA_INT + "train_features_clean_stats.parquet.gzip")
    df_wiki_test = pd.read_parquet(PATH_DATA_INT + "test_features_clean_stats.parquet.gzip")

Not a Goolge Drive Environment. Loading local files.


In [141]:
df_wiki_train.shape
df_wiki_test.shape

(416768, 37)

(119092, 38)

# 4.0 Data Cleaning and Feature Engineering

**Tips on Creating Features**
- Linear models learn sums and differences naturally, but can't learn anything more complex.
- Ratios seem to be difficult for most models to learn. Ratio combinations often lead to some easy performance gains.
- Linear models and neural nets generally do better with normalized features. Neural nets especially need features scaled to values not too far from 0. Tree-based models (like random forests and XGBoost) can sometimes benefit from normalization, but usually much less so.
- Tree models can learn to approximate almost any combination of features, but when a combination is especially important they can still benefit from having it explicitly created, especially when data is limited.
- Counts are especially helpful for tree models, since these models don't have a natural way of aggregating information across many features at once.
[Source](https://www.kaggle.com/code/ryanholbrook/creating-features)

### Calculate avg word2vec Features (w2v_)

## Gensim

In [142]:
%%time
# Gensim
wv = gensim_api.load('word2vec-google-news-300')

CPU times: user 22.5 s, sys: 1.78 s, total: 24.3 s
Wall time: 27.8 s


In [143]:
lemmatizer=WordNetLemmatizer()

def create_lemma(text_list):
    text_list = [lemmatizer.lemmatize(word) for word in text_list]
    return text_list

def avg_word2vec(sentence_list, model):
    result = np.mean(np.mean([model.wv[word] for word in sentence_list if word in model.wv.index_to_key], axis=0))
    return result.round(5)


def ft_avg_word2vec(sentence_list, model):
    result= np.mean(np.mean([model.get_word_vector(word) for word in sentence_list]))
    return result.round(5)


def ft_avg_sent2vec(sentence_list, model):
    result = np.mean(model.get_sentence_vector(sentence_list))
    return result.round(5)

In [144]:
%%time
df_wiki_train['tokenized'] = df_wiki_train['cleaned_text'].progress_apply(nltk.word_tokenize)
df_wiki_test['tokenized'] = df_wiki_test['cleaned_text'].progress_apply(nltk.word_tokenize)
df_wiki_train['tokenized_lemma'] = df_wiki_train['tokenized'].progress_apply(create_lemma)
df_wiki_test['tokenized_lemma'] = df_wiki_test['tokenized'].progress_apply(create_lemma)

100%|██████████| 119092/119092 [00:04<00:00, 27210.62it/s]

CPU times: user 52.6 s, sys: 8.91 s, total: 1min 1s
Wall time: 1min 9s


In [145]:
# Only use English tokenized lemma
df_wiki_train_en = df_wiki_train[df_wiki_train['stats_language_code'] == 'en'].copy()
sentences_wiki_train_en = df_wiki_train_en['tokenized_lemma'].to_list()

df_wiki_test_en = df_wiki_test[df_wiki_test['stats_language_code'] == 'en'].copy()
sentences_wiki_test_en = df_wiki_test_en['tokenized_lemma'].to_list()

In [146]:
df_wiki_train_nonen = df_wiki_train[df_wiki_train['stats_language_code'] != 'en'].copy()
df_wiki_test_nonen = df_wiki_test[df_wiki_test['stats_language_code'] != 'en'].copy()

In [147]:
%%time
# Lets train Word2vec model
word2vec_model_wiki_train_en = gensim.models.Word2Vec(sentences_wiki_train_en)

CPU times: user 1min 9s, sys: 1.66 s, total: 1min 11s
Wall time: 25.7 s


In [148]:
%%time
# Lets train Word2vec model
word2vec_model_wiki_test_en = gensim.models.Word2Vec(sentences_wiki_test_en)

CPU times: user 18.8 s, sys: 239 ms, total: 19.1 s
Wall time: 6.89 s


In [149]:
%%time
df_wiki_train_en['w2v_gen_avg_word2vec_mean'] = df_wiki_train_en['tokenized_lemma'].progress_apply(
    lambda x: avg_word2vec(x, word2vec_model_wiki_train_en))

df_wiki_test_en['w2v_gen_avg_word2vec_mean'] = df_wiki_test_en['tokenized_lemma'].progress_apply(
    lambda x: avg_word2vec(x, word2vec_model_wiki_test_en))

100%|██████████| 117060/117060 [01:17<00:00, 1501.41it/s]

CPU times: user 8min 56s, sys: 8.5 s, total: 9min 5s
Wall time: 9min 8s


In [150]:
df_wiki_train_nonen['w2v_gen_avg_word2vec_mean'] = -1
df_wiki_test_nonen['w2v_gen_avg_word2vec_mean'] = -1

In [151]:
# %%time
# #df_wiki_train['tokenized_lemma'] = df_wiki_train['tokenized_lemma'].fillna(-1)
# df_wiki_train['w2v_gen_avg_word2vec_mean']= np.where(
#                             df_wiki_train['stats_language_code'] == 'en',
#                             df_wiki_train['tokenized_lemma'].apply(lambda x: avg_word2vec(x, word2vec_model_wiki_train),
#                             -1))

In [152]:
# %%time
# #df_wiki_test['tokenized_lemma'] = df_wiki_test['tokenized_lemma'].fillna(-1)
# df_wiki_test['w2v_gen_avg_word2vec_mean']= np.where(
#                         df_wiki_test['stats_language_code'] == 'en',
#                         df_wiki_test['tokenized_lemma'].apply(lambda x: avg_word2vec(x, word2vec_model_wiki_test),
#                         -1))

## Fasttext

In [153]:
import fasttext
import fasttext.util
#fasttext.util.download_model('en', if_exists='ignore')  # English

In [154]:
# https://fasttext.cc/docs/en/crawl-vectors.html
ft_model = fasttext.load_model('../models/fasttext_cc.en.300.bin')
#ft_model = fasttext.load_model(PATH_MOD+fn_ft_model)

In [155]:
df_wiki_train_en['w2v_ft_sen2vec_mean'] = df_wiki_train_en['cleaned_text'].progress_apply(
    lambda x: ft_avg_sent2vec(x, ft_model))
df_wiki_test_en['w2v_ft_sen2vec_mean'] = df_wiki_test_en['cleaned_text'].progress_apply(
    lambda x: ft_avg_sent2vec(x, ft_model))

100%|██████████| 117060/117060 [00:08<00:00, 13186.95it/s]


In [156]:
df_wiki_train_en['w2v_ft_word2vec_mean'] = df_wiki_train_en['cleaned_text'].progress_apply(
    lambda x: ft_avg_word2vec(x, ft_model))
df_wiki_test_en['w2v_ft_word2vec_mean'] = df_wiki_test_en['cleaned_text'].progress_apply(
    lambda x: ft_avg_word2vec(x, ft_model))

100%|██████████| 117060/117060 [00:35<00:00, 3333.69it/s]


In [157]:
df_wiki_train_nonen['w2v_ft_sen2vec_mean'] = -1
df_wiki_test_nonen['w2v_ft_sen2vec_mean'] = -1

df_wiki_train_nonen['w2v_ft_word2vec_mean'] = -1
df_wiki_test_nonen['w2v_ft_word2vec_mean'] = -1

In [158]:
# df_wiki_train['w2v_ft_sen2vec_mean'] = df_wiki_train['cleaned_text'].progress_apply(
#     lambda x: ft_avg_sent2vec(x, ft_model))
# df_wiki_test['w2v_ft_sen2vec_mean'] = df_wiki_test['cleaned_text'].progress_apply(
#     lambda x: ft_avg_sent2vec(x, ft_model))

In [159]:
# %%time
# df_wiki_train['w2v_ft_sen2vec_mean'] = np.where(
#                                     df_wiki_train['stats_language_code'] == 'en',
#                                     df_wiki_train['cleaned_text'].apply(lambda x: ft_avg_sent2vec(x, ft_model),
#                                     -1))

# df_wiki_test['w2v_ft_sen2vec_mean'] = np.where(
#                                     df_wiki_test['stats_language_code'] == 'en',
#                                     df_wiki_test['cleaned_text'].apply(lambda x: ft_avg_sent2vec(x, ft_model),
#                                     -1))

In [160]:
# df_wiki_train['w2v_ft_word2vec_mean'] = df_wiki_train['cleaned_text'].progress_apply(
#     lambda x: ft_avg_word2vec(x, ft_model))
# df_wiki_test['w2v_ft_word2vec_mean'] = df_wiki_test['cleaned_text'].progress_apply(
#     lambda x: ft_avg_word2vec(x, ft_model))

In [161]:
# %%time
# df_wiki_train['w2v_ft_word2vec_mean'] = np.where(
#                                         df_wiki_train['stats_language_code'] == 'en',
#                                         df_wiki_train['cleaned_text'].apply(lambda x: ft_avg_word2vec(x, ft_model),
#                                         -1))

# df_wiki_test['w2v_ft_word2vec_mean'] = np.where(
#                                         df_wiki_test['stats_language_code'] == 'en',
#                                         df_wiki_test['cleaned_text'].apply(lambda x: ft_avg_word2vec(x, ft_model),
#                                         -1))

***


In [177]:
df_wiki_train = pd.DataFrame()
df_wiki_train = pd.concat([df_wiki_train_en, df_wiki_train_nonen], axis =0).sort_index()

df_wiki_test = pd.DataFrame()
df_wiki_test = pd.concat([df_wiki_test_en, df_wiki_test_nonen], axis =0).sort_index()

In [180]:
df_wiki_train.fillna(-1, inplace=True)
df_wiki_test.fillna(-1, inplace=True)

In [181]:
df_wiki_train.shape
df_wiki_test.shape

(416768, 42)

(119092, 43)

In [182]:
df_wiki_train.head(4)

,original_text,cleaned_text,label,stats_lrb_count,stats_rrb_count,stats_comma_count,stats_equalsign_count,stats_char_count,stats_image_description,stats_frac_description,stats_file_description,stats_formula_description,stats_language_code,stats_avg_char_per_word,stats_word_count,stats_letter_count,stats_long_word_count,stats_syllable_count,stats_polysyllab_count,stats_monosyllab_count,stats_reading_time,stats_avg_letter_per_word,stats_sentence_length,stats_syllable_avg,stats_mini_word_count,stats_long_numbers_count,stats_lexical_diversity,stats_frac_word_comma,stats_frac_mini_word,stats_frac_long_word,stats_frac_monosyllable,stats_frac_polysyllable,stats_max_chars_word,stats_avg_chars_word,stats_max_syllables_word,stats_avg_syllables_word,stats_language_no,tokenized,tokenized_lemma,w2v_gen_avg_word2vec_mean,w2v_ft_sen2vec_mean,w2v_ft_word2vec_mean
0,"There is manuscript evidence that Austen continued to work on these pieces as late as the period 1809 â '' 11 , and that her niece and nephew , Anna and James Edward Austen , made further additions as late as 1814 .",there is manuscript evidence that austen continued to work on these pieces as late as the period 1809 11 and that her niece and nephew anna and james edward austen made further additions as late as 1814,1,0,0,3,0,173,0,0,0,0,en,4.5,37,166,5,50,3,27,2.4,4.5,37.0,1.4,12,0,0.8,0.1,0.3,0.1,0.7,0.1,10,4.5,3,1.4,18,"[there, is, manuscript, evidence, that, austen, continued, to, work, on, these, pieces, as, late, as, the, period, 1809, 11, and, that, her, niece, and, nephew, anna, and, james, edward, austen, made, further, additions, as, late, as, 1814]","[there, is, manuscript, evidence, that, austen, continued, to, work, on, these, piece, a, late, a, the, period, 1809, 11, and, that, her, niece, and, nephew, anna, and, james, edward, austen, made, further, addition, a, late, a, 1814]",-0.096,-0.000,0.005
1,"In a remarkable comparative analysis , Mandaean scholar Säve-Söderberg demonstrated that Mani 's Psalms of Thomas were closely related to Mandaean texts .",in a remarkable comparative analysis mandaean scholar saeve soederberg demonstrated that mani psalms of thomas were closely related to mandaean texts,1,0,0,1,0,132,0,0,0,0,en,6.1,21,129,10,34,3,12,1.9,6.1,21.0,1.6,4,0,1.0,0.0,0.2,0.5,0.6,0.1,12,6.1,4,1.6,18,"[in, a, remarkable, comparative, analysis, mandaean, scholar, saeve, soederberg, demonstrated, that, mani, psalms, of, thomas, were, closely, related, to, mandaean, texts]","[in, a, remarkable, comparative, analysis, mandaean, scholar, saeve, soederberg, demonstrated, that, mani, psalm, of, thomas, were, closely, related, to, mandaean, text]",-0.053,-0.001,0.003
2,"Before Persephone was released to Hermes , who had been sent to retrieve her , Hades tricked her into eating pomegranate seeds , -LRB- six or three according to the telling -RRB- which forced her to return to the underworld for a period each year .",before persephone was released to hermes who had been sent to retrieve her hades tricked her into eating pomegranate seeds six or three according to the telling which forced her to return to the underworld for a period each year,1,1,1,3,0,203,0,0,0,0,en,4.7,40,189,8,52,2,30,2.8,4.7,40.0,1.3,17,0,0.8,0.1,0.4,0.2,0.8,0.0,11,4.7,3,1.3,18,"[before, persephone, was, released, to, hermes, who, had, been, sent, to, retrieve, her, hades, tricked, her, into, eating, pomegranate, seeds, six, or, three, according, to, the, telling, which, forced, her, to, return, to, the, underworld, for, a, period, each, year]","[before, persephone, wa, released, to, hermes, who, had, been, sent, to, retrieve, her, hades, tricked, her, into, eating, pomegranate, seed, six, or, three, according, to, the, telling, which, forced, her, to, return, to, the, underworld, for, a, period, each, year]",-0.114,-0.001,0.003
3,"Cogeneration plants are commonly found in district heating systems of cities , hospitals , prisons , oil refineries , paper mills , wastewater treatment plants , thermal e

# Export

In [183]:
columns = df_wiki_train.columns.to_list()
feature_columns = sorted([x for x in columns if x.startswith("w2v_")])

feature_columns

['w2v_ft_sen2vec_mean', 'w2v_ft_word2vec_mean', 'w2v_gen_avg_word2vec_mean']

In [185]:
# Export AoA features
df_export = df_wiki_train[feature_columns]
df_export.to_csv(PATH_DATA_INT + "train_features_w2v.csv", index=False)
df_export.to_parquet(PATH_DATA_INT + "train_features_w2v.parquet.gzip", compression='gzip')

df_export = df_wiki_test[feature_columns]
df_export.to_csv(PATH_DATA_INT + "test_features_w2v.csv", index=False)
df_export.to_parquet(PATH_DATA_INT + "test_features_w2v.parquet.gzip", compression='gzip')

# Watermark

In [186]:
%watermark

Last updated: 2023-02-13T17:49:39.645487+01:00

Python implementation: CPython
Python version       : 3.9.15
IPython version      : 8.8.0

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 22.2.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [187]:
%watermark --iversions

re      : 2.2.1
gensim  : 4.3.0
nltk    : 3.8.1
platform: 1.0.8
fasttext: 0.9.2
numpy   : 1.23.5
seaborn : 0.12.2
pandas  : 1.5.3



In [188]:
t_end = time.time()
total_runtime = t_end-t_start
total_runtime_min = round((total_runtime/60),2)
print(str(total_runtime_min)+" minutes")

22.13 minutes


In [189]:
send_push(f"Calculate word2vec features finished in: {total_runtime_min} min.")